# Proof of concept
Small notebook illustrating how DHFRs change their appearance according to the type of editing executed on the spliced area.

## Libraries importing

In [ ]:
import os
import sys
sys.path.append('..')
from isplutils.network import DnCNN
import glob
import cv2
import numpy as np
import keras
import tensorflow as tf
import rasterio
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (7, 7)
%matplotlib inline

## Execution parameters

In [ ]:
data_dir = '../data/poc' # directory containing the sample images used in the paper
gpu = 3 # GPU ID to use
model_dir = '../weights/'
sample_paths = glob.glob(os.path.join(data_dir, '*.tiff'))
sample_paths = {os.path.basename(path).split('.')[0]: path for path in sample_paths}

### Model loading

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = f'{gpu}'  # set the GPU device

print('tf    version:', tf.__version__)
print('keras version:', keras.__version__)

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

configSess = tf.ConfigProto()
# Allowing the GPU memory to grow avoid preallocating all the GPU memory
configSess.gpu_options.allow_growth = True
set_session(tf.Session(config=configSess))

# define the network and load weights
model = DnCNN(model_path=model_dir)

## Let's load the images and extract the DHFRs
Test images are VV-polarization GRD Sentinel-1 samples, that have been normalized using a simple multiplicative scaling (normalization strategies of the original paper) and converted to 8-bit to ease of use.  

In [ ]:
import sklearn

fsize = 20  # fontsize

sorted_editing = ['AverageBlur', 'MedianBlur', 'RotationResize', 'AdditiveWhiteGaussianNoise', 'AdditiveLaplacianNoise', 'SpeckleNoise']
for editing in sorted_editing:

    # --- Load the sample path using rasterio
    path = sample_paths[editing]
    with rasterio.open(path, 'r') as src:
        img = np.squeeze(src.read())

    # --- Extract the DHFR
    img_float = img.astype(np.float32)
    img_float = (img_float-img_float.min())/(img_float.max()-img_float.min()) # simple min-max scaling between 0-1
    img_float = img_float[np.newaxis, :, :, np.newaxis] # add batch dimension
    dhfr = model.predict(img_float)

    # --- Plot images and DHFRs, together with the ROC curve to illustrate how the appearance of the DHFR influences the AUC value

    # Load the reference mask
    mask = cv2.imread(os.path.join(data_dir, 'mask.png'), cv2.IMREAD_UNCHANGED)
    mask = cv2.cvtColor(mask, cv2.COLOR_RGBA2GRAY)

    # Compute the ROC curve ad AUC
    fpr, tpr, _ = sklearn.metrics.roc_curve(mask.ravel(), dhfr.ravel(), pos_label=255)
    auc = np.trapz(tpr, fpr)

    # Normalize the DHFR for visualization
    dhfr = np.squeeze(((dhfr-dhfr.min())/(dhfr.max()-dhfr.min())))
    dhfr = (dhfr*255).astype(np.uint8)

    # Prepare the plot
    plt.figure(figsize=(25,23))
    plt.suptitle(f'DHFR for {editing} editing', y=0.65, fontsize=fsize)
    plt.subplot(141)
    plt.imshow(img, cmap='gray', clim=[0, 255])
    plt.title('Input image', fontsize=fsize-5)
    plt.subplot(142)
    plt.imshow(dhfr, cmap='gray')
    plt.title('DHFR', fontsize=fsize-5)
    plt.subplot(143)
    plt.imshow(mask, cmap='gray')
    plt.title('Ground truth', fontsize=fsize-5)
    plt.subplot(144)
    plt.plot(fpr, tpr, label=f"AUC score {auc:.3f}")
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.xlabel('FPR', fontsize=fsize-7)
    plt.ylim([0.0, 1.05])
    plt.ylabel('TPR', fontsize=fsize-7)
    plt.grid()
    plt.axis("image")
    plt.legend(fontsize=fsize-7)
    plt.title('ROC curve', fontsize=fsize-5)
    plt.show()

## Let's highlight and zoom-in in the tampered areas

In [ ]:
sorted_editing = ['AverageBlur', 'MedianBlur', 'RotationResize', 'AdditiveWhiteGaussianNoise', 'AdditiveLaplacianNoise', 'SpeckleNoise']
for editing in sorted_editing:

    # --- Load the sample path using rasterio
    path = sample_paths[editing]
    with rasterio.open(path, 'r') as src:
        img = np.squeeze(src.read())

    # --- Extract the DHFR
    img_float = img.astype(np.float32)
    img_float = (img_float-img_float.min())/(img_float.max()-img_float.min()) # simple min-max scaling between 0-1
    img_float = img_float[np.newaxis, :, :, np.newaxis] # add batch dimension
    dhfr = model.predict(img_float)

    # --- Plot images and DHFRs, together with the ROC curve to illustrate how the appearance of the DHFR influences the AUC value

    # Load the reference mask
    mask = cv2.imread(os.path.join(data_dir, 'mask.png'), cv2.IMREAD_UNCHANGED)
    mask = cv2.cvtColor(mask, cv2.COLOR_RGBA2GRAY)

    # Normalize the DHFR for visualization
    dhfr = np.squeeze(((dhfr-dhfr.min())/(dhfr.max()-dhfr.min())))
    dhfr = (dhfr*255).astype(np.uint8)

    # Find the mask countours
    mask = mask > 200
    mask = (mask.astype(np.uint8))*255
    edges = cv2.Canny(mask, 100, 200)
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)


    # Prepare the plot
    plt.figure(figsize=(25,23))
    plt.suptitle(f'DHFR for {editing} editing', y=0.65, fontsize=fsize)
    plt.subplot(131)
    plt.imshow(img, cmap='gray', clim=[0, 255])
    plt.plot(contours[-1][:, :, 0].flatten(), contours[-1][:, :, 1].flatten(), color='red')
    plt.title('Input image', fontsize=fsize-5)
    plt.subplot(132)
    plt.imshow(dhfr, cmap='gray')
    plt.title('DHFR', fontsize=fsize-5)
    plt.plot(contours[-1][:, :, 0].flatten(), contours[-1][:, :, 1].flatten(), color='red')
    plt.subplot(133)
    plt.imshow(mask, cmap='gray')
    plt.title('Ground truth', fontsize=fsize-5)
    plt.show()

In [ ]:
import pandas as pd
df = pd.read_pickle('test_imgs/nice_images_quickshift/df_nice_looking_images.pkl')

for i, r in df.iterrows():
    # Load the sample
    sample = cv2.imread(os.path.join('test_imgs/nice_images_quickshift', r['forged_image']), cv2.IMREAD_UNCHANGED)
    
    # Plot the image with contour
    plt.imshow(sample, cmap='gray')
    plt.plot(contours[-1][:, :, 0].flatten(), contours[-1][:, :, 1].flatten(), color='red')
    plt.axis('off')
    plt.savefig(os.path.join('test_imgs/nice_images_quickshift', f"{r['forged_image'].split('.png')[0]}_tamp.png"),
                bbox_inches='tight')
    plt.show()

### Let's also save the close-up

In [ ]:
import pandas as pd
df = pd.read_pickle('test_imgs/nice_images_quickshift/df_nice_looking_images.pkl')

for i, r in df.iterrows():
    # Load the sample
    sample = cv2.imread(os.path.join('test_imgs/nice_images_quickshift', r['forged_image']), cv2.IMREAD_UNCHANGED)
    print(r['forged_image'])
    # Plot the image with contour
    edges = cv2.Canny(mask[450:750, 250:550], 100, 200)
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    plt.imshow(sample[450:750, 250:550], cmap='gray')
    plt.plot(contours[-1][:, :, 0].flatten(), contours[-1][:, :, 1].flatten(), color='red')
    plt.axis('off')
    plt.savefig(os.path.join('test_imgs/nice_images_quickshift', f"{r['forged_image'].split('.png')[0]}_tamp_closeup.png"),
                bbox_inches='tight')
    plt.show()

### And also repeat for the fingerprints

In [ ]:
for i, r in df.iterrows():
    # Load the sample
    sample = cv2.imread(os.path.join('test_imgs/nice_images_quickshift', r['forged_noiseprint']), cv2.IMREAD_UNCHANGED)
    
    # Plot the image with contour
    edges = cv2.Canny(mask, 100, 200)
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    plt.imshow(sample, cmap='gray', clim=[0, 255])
    plt.plot(contours[-1][:, :, 0].flatten(), contours[-1][:, :, 1].flatten(), color='red')
    plt.axis('off')
    plt.savefig(os.path.join('test_imgs/nice_images_quickshift', f"{r['forged_noiseprint'].split('.png')[0]}_tamp.png"),
                bbox_inches='tight')
    plt.show()

In [ ]:
for i, r in df.iterrows():
    # Load the sample
    sample = cv2.imread(os.path.join('test_imgs/nice_images_quickshift', r['forged_noiseprint']), cv2.IMREAD_UNCHANGED)
    sample = (sample-sample.min())/(sample.max()-sample.min())
    print(f'{r["forged_image"]}, {sample.min()}, {sample.max()}')
    # Plot the image with contour
    edges = cv2.Canny(mask[450:750, 250:550], 100, 200)
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    plt.imshow(sample[450:750, 250:550], cmap='gray', clim=[sample[450:750, 250:550].min(), sample[450:750, 250:550].max()])
    plt.plot(contours[-1][:, :, 0].flatten(), contours[-1][:, :, 1].flatten(), color='red')
    plt.axis('off')
    plt.savefig(os.path.join('test_imgs/nice_images_quickshift', f"{r['forged_noiseprint'].split('.png')[0]}_tamp_closeup.png"),
                bbox_inches='tight')
    plt.show()